# Verify hyperboloid
> Is the hyperboloid projection script doing what it should?

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
import numpy as np

sys.path.append("/home/phil/mixture_embeddings")
from src.embeddings.mixture_embeddings import (
    get_mixture_embeddings,
    convert_geometry_after,
)


INFO: Using numpy backend


In [5]:
# Rejection sample some points from 6-D poincare disk

points = np.random.rand(100000, 6)
mask = np.sum(points ** 2, axis=1) < 1
points = points[mask]

# Convert to hyperboloid
def to_hyperboloid_points(poincare_pts):
    """
    Post: result.shape[1] == poincare_pts.shape[1] + 1
    """
    norm_sqd = (poincare_pts ** 2).sum(axis=1)
    N = poincare_pts.shape[1]
    result = np.zeros((poincare_pts.shape[0], N + 1), dtype=np.float64)
    result[:, :N] = (2.0 / (1 - norm_sqd))[:, np.newaxis] * poincare_pts
    result[:, N] = (1 + norm_sqd) / (1 - norm_sqd)
    return result


hyperboloid_points = to_hyperboloid_points(points)


In [6]:
# Assert 7-dimensional

hyperboloid_points.shape


(8039, 7)

In [7]:
# Assert Q(x) = -1 for all x in X


def minkowski_norm(hyperboloid_pts):
    """
    Compute Minkowski norm for points in the hyperboloid model.
    """
    N = hyperboloid_pts.shape[1] - 1
    return (
        np.sum(hyperboloid_pts[:, :N] ** 2, axis=1) - hyperboloid_pts[:, N] ** 2
    )


minkowski_norm(hyperboloid_points)


array([-1., -1., -1., ..., -1., -1., -1.])

In [8]:
# Great, that worked. Do our mixture embeddings follow this?

import anndata

adata = anndata.read_h5ad("data/big_table_with_embeddings.h5ad")


In [13]:
# Frechet means don't break this - that's good. However, I messed up the index
# so I need to redo everything with time-like dimension LAST.

for key in adata.obsm:
    if str(key).startswith("hyp"):
        norms = minkowski_norm(adata.obsm[key])
        print(
            key,
            adata.obsm[key].shape[1],
            np.min(norms),
            np.max(norms),
            np.allclose(norms, -1.0),
            sep="\t",
        )


hyp_mix_128	129	-1.0000000000000016	-0.9999999999999984	True
hyp_mix_16	17	-1.0000000000000016	-0.9999999999999986	True
hyp_mix_2	3	-1.0000000002910383	-0.9999999997671694	True
hyp_mix_2_PAC	2	-0.9977814849190494	0.9979777817533649	False
hyp_mix_32	33	-1.0000000000000016	-0.9999999999999984	True
hyp_mix_4	5	-1.0000000000022737	-0.9999999999972715	True
hyp_mix_64	65	-1.0000000000000016	-0.9999999999999984	True
hyp_mix_8	9	-1.0000000000000355	-0.9999999999999645	True
